# IBM watsonx.ai

>WatsonxToolkit是IBM [watsonx.ai](https://www.ibm.com/products/watsonx-ai) Toolkit的封装。

此示例演示如何使用`LangChain`来使用`watsonx.ai toolkit`。

## 概述

### 集成详情

| 类 | 包 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/toolkits/ibm/) | 包下载量 | 最新包版本 |
| :--- | :--- | :---: | :---: | :---: | :---: |
| [WatsonxToolkit](https://python.langchain.com/api_reference/ibm/toolkit/langchain_ibm.toolkit.WatsonxToolkit.html) | [langchain-ibm](https://python.langchain.com/api_reference/ibm/index.html) | ❌ | ✅ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-ibm?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-ibm?style=flat-square&label=%20) |

## 设置

要访问 IBM watsonx.ai 工具包，您需要创建一个 IBM watsonx.ai 账户，获取 API 密钥，并安装 `langchain-ibm` 集成包。

### 认证信息

此单元格定义了使用 watsonx Toolkit 所需的 WML 认证信息。

**操作：** 提供 IBM Cloud 用户 API 密钥。有关详细信息，请参阅
[文档](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui)。

In [1]:
import os
from getpass import getpass

watsonx_api_key = getpass()
os.environ["WATSONX_APIKEY"] = watsonx_api_key

此外，您还可以将其他秘密作为环境变量传递。

In [ ]:
import os

os.environ["WATSONX_URL"] = "your service instance url"
os.environ["WATSONX_TOKEN"] = "your token for accessing the service instance"

### 安装

LangChain IBM 集成位于 `langchain-ibm` 包中：

In [ ]:
!pip install -qU langchain-ibm

## 实例化

初始化 `WatsonxToolkit` 类。

In [2]:
from langchain_ibm import WatsonxToolkit

watsonx_toolkit = WatsonxToolkit(
    url="https://us-south.ml.cloud.ibm.com",
)

对于某些需求，可以选择将 IBM 的 [`APIClient`](https://ibm.github.io/watsonx-ai-python-sdk/base.html#apiclient) 对象传递给 `WatsonxToolkit` 类。

In [ ]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(...)

watsonx_toolkit = WatsonxToolkit(
    watsonx_client=api_client,
)

## 工具

### 获取所有工具

可以通过一个 `WatsonxTool` 对象列表来获取所有可用的工具。

In [ ]:
watsonx_toolkit.get_tools()


[WatsonxTool(name='GoogleSearch', description='Search for online trends, news, current events, real-time information, or research topics.', args_schema=<class 'langchain_ibm.toolkit.ToolArgsSchema'>, agent_description='Search for online trends, news, current events, real-time information, or research topics.', tool_config_schema={'title': 'config schema for GoogleSearch tool', 'type': 'object', 'properties': {'maxResults': {'title': 'Max number of results to return', 'type': 'integer', 'minimum': 1, 'maximum': 20}}}, watsonx_client=<ibm_watsonx_ai.client.APIClient object at 0x127e0f490>),
 WatsonxTool(name='WebCrawler', description='Useful for when you need to summarize a webpage. Do not use for Web search.', args_schema=<class 'langchain_ibm.toolkit.ToolArgsSchema'>, agent_description='Useful for when you need to summarize a webpage. Do not use for Web search.', tool_input_schema={'type': 'object', 'properties': {'url': {'title': 'url', 'description': 'URL for the webpage to be scrap

### 获取工具
您也可以通过名称获取特定的 `WatsonxTool`。

In [4]:
google_search = watsonx_toolkit.get_tool(tool_name="GoogleSearch")

## 调用

### 调用工具并输入简单内容

In [5]:
search_result = google_search.invoke(input="IBM")
search_result

{'output': '[{"title":"IBM - United States","description":"Technology & Consulting. From next-generation AI to cutting edge hybrid cloud solutions to the deep expertise of IBM Consulting, IBM has what it takes to help\xa0...","url":"https://www.ibm.com/us-en"},{"title":"IBM - Wikipedia","description":"International Business Machines Corporation (using the trademark IBM), nicknamed Big Blue, is an American multinational technology company headquartered in\xa0...","url":"https://en.wikipedia.org/wiki/IBM"},{"title":"IBM Envizi ESG Suite","description":"Envizi systemizes the capture, transformation and consolidation of disparate sustainability data into a single source of truth and delivers actionable insights.","url":"https://www.ibm.com/products/envizi"},{"title":"IBM Research","description":"Tools + Code · BeeAI Framework. Open-source framework for building, deploying, and serving powerful agentic workflows at scale. · Docling. An open-source tool\xa0...","url":"https://research.ibm.co

要获取已收到的结果列表，您可以执行以下单元格。

In [ ]:
import json

output = json.loads(search_result.get("output"))
output

### 使用配置调用工具

您可以通过查看工具的 `tool_config_schema` 来检查工具是否具有配置模式并查看其属性。

在此示例中，该工具具有一个配置模式，其中包含 `maxResults` 参数，用于设置返回结果的最大数量。

In [7]:
google_search.tool_config_schema

{'title': 'config schema for GoogleSearch tool',
 'type': 'object',
 'properties': {'maxResults': {'title': 'Max number of results to return',
   'type': 'integer',
   'minimum': 1,
   'maximum': 20}}}

要设置 `tool_config` 参数，您需要使用 `set_tool_config()` 方法，并根据上面的 `tool_config_schema` 传递正确的 `dict`。

In [8]:
import json

config = {"maxResults": 3}
google_search.set_tool_config(config)

search_result = google_search.invoke(input="IBM")
output = json.loads(search_result.get("output"))

最多应该有 3 个结果。

In [9]:
print(len(output))

3


### 使用输入模式调用工具

我们需要另外一个工具（带有一个输入模式）用于示例目的。

In [10]:
weather_tool = watsonx_toolkit.get_tool("Weather")

要检查工具是否具有输入 schema 并查看其属性，您可以查看工具的 `tool_input_schema`。

在此示例中，该工具的输入 schema 包含一个必需参数和一个可选参数。

In [11]:
weather_tool.tool_input_schema

{'type': 'object',
 'properties': {'location': {'title': 'location',
   'description': 'Name of the location',
   'type': 'string'},
  'country': {'title': 'country',
   'description': 'Name of the state or country',
   'type': 'string'}},
 'required': ['location']}

要正确地将输入传递给 `invoke()`，你需要创建一个 `invoke_input` 字典，其中包含必需的参数作为键，以及它们的值。

In [12]:
invoke_input = {
    "location": "New York",
}

weather_result = weather_tool.invoke(input=invoke_input)
weather_result

{'output': 'Current weather in New York:\nTemperature: 0°C\nRain: 0mm\nRelative humidity: 63%\nWind: 7.6km/h\n'}

本次输出的是单个字符串值。您可以执行以下单元格来获取并打印它。

In [13]:
output = weather_result.get("output")
print(output)

Current weather in New York:
Temperature: 0°C
Rain: 0mm
Relative humidity: 63%
Wind: 7.6km/h



### 使用 ToolCall 调用工具

我们也可以使用 ToolCall 来调用工具，在这种情况下将返回一个 ToolMessage：

In [14]:
invoke_input = {
    "location": "Los Angeles",
}
tool_call = dict(
    args=invoke_input,
    id="1",
    name=weather_tool.name,
    type="tool_call",
)
weather_tool.invoke(input=tool_call)

ToolMessage(content='{"output": "Current weather in Los Angeles:\\nTemperature: 8.6°C\\nRain: 0mm\\nRelative humidity: 61%\\nWind: 8.4km/h\\n"}', name='Weather', tool_call_id='1')

## 在 Agent 中使用

In [ ]:
from langchain_ibm import ChatWatsonx

llm = ChatWatsonx(
    model_id="meta-llama/llama-3-3-70b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
)

In [16]:
from langgraph.prebuilt import create_react_agent

tools = [weather_tool]
agent = create_react_agent(llm, tools)

In [17]:
example_query = "What is the weather in Boston?"

events = agent.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the weather in Boston?
================================== Ai Message ==================================
Tool Calls:
  Weather (chatcmpl-tool-6a6c21402c824e43bdd2e8ba390af4a8)
 Call ID: chatcmpl-tool-6a6c21402c824e43bdd2e8ba390af4a8
  Args:
    location: Boston
================================= Tool Message =================================
Name: Weather

{"output": "Current weather in Boston:\nTemperature: -1°C\nRain: 0mm\nRelative humidity: 53%\nWind: 8.3km/h\n"}
================================== Ai Message ==================================

The current weather in Boston is -1°C with 0mm of rain, a relative humidity of 53%, and a wind speed of 8.3km/h.


## API 参考

有关所有 `WatsonxToolkit` 功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/ibm/toolkit/langchain_ibm.toolkit.WatsonxToolkit.html)。